In [1]:
import pandas as pd
import json
from pandas import Series,DataFrame
import re
import datetime
from bs4 import BeautifulSoup
import requests
#寻找小区对应的区域和建造年份
def get_data(x):
    try:
        html=requests.get('https://dl.lianjia.com/xiaoqu/rs'+x)
        soup=BeautifulSoup(html.text,'lxml')
        s=soup.select('.info .title a')
        for i in range(0,len(s)):
            if s[i].string==x:
                district=soup.select('.info .positionInfo .district')[i].string
                bizcircle=soup.select('.info .positionInfo .bizcircle')[i].string
                temp=soup.select('.info .positionInfo')[i].get_text()
                nianfen=re.search('\d+',temp).group()
                return x+' '+district+' '+bizcircle+' '+ str(nianfen)
        else:
            return None
    except IndexError as e:
        print(x)

pd.set_option('display.max_rows',None)
# #设置显示全部列，不省略
pd.set_option('display.max_columns',None)

item=[]
for line in open('data/lianjia.json','r',encoding='utf-8'):
    item.append(json.loads(line))
    
df=DataFrame(item)
df.drop_duplicates(inplace=True)
#把车位这一行删掉
df.drop(index=df[df['area']=='车位'].index,inplace=True) 

In [2]:
getarea=lambda x: float(re.search(r"\d+(\.\d+)?",x).group())
df['area']=df['area'].apply(getarea)
tofloat=lambda x:float(x)
df['unitPrice']=df['unitPrice'].apply(tofloat)
df['totalPrice']=df['totalPrice'].apply(tofloat)
df['dealDate']=pd.to_datetime(df['dealDate'])
df['dealCycle']=df['dealCycle'].map(lambda x:int(re.search('\d+',x).group()))
df['guaPai']=df['guaPai'].apply(getarea)

In [3]:
#找出降价幅度大的房子
f=df[((df['guaPai']-df['totalPrice'])/df['guaPai']>0.05) & (df['dealDate'].map(lambda x:x.year)==2021)].sort_values(by=['title','dealDate'])
f['chazhi']=round((f['guaPai']-f['totalPrice'])/f['guaPai'],2)

In [4]:
#找出各个销售的销售量，目前看工人村单个销售卖的还真多
s=df.groupby(['seller'])[['seller']].count()
s.index.name='num'
s.sort_values(by=['seller'],ascending=False)
s.head()

,seller
num,
SSC业务运营中心,17
丁一航,1
丁丽君,1
丁伟洧,2
丁余艳,14


In [5]:
#找出各小区的销售数量，先忽略时间
# xiaoqu=df.groupby(['title'])[['title']].count()
xiaoqu=df['title'].groupby([df['title'],df['dealDate'].apply(lambda x:x.year)]).count()
xiaoqu.index.names=['title','year']
result=list(set(list(map(lambda x:re.search('悦泰(\w+)?',x),xiaoqu.index.levels[0]))))
result.remove(None)
r=[i.group() for i in result]


In [6]:
s1=xiaoqu.loc[r[1]]
s2=xiaoqu.loc[r[0]]
s1.name=r[1]
s2.name=r[0]
print(s1,s2)


year
2016    2
2019    8
2020    6
2021    8
Name: 悦泰福里, dtype: int64 year
2016    12
2017     9
2018     8
2019    28
2020    23
2021     9
Name: 悦泰山里, dtype: int64


In [7]:
#找出某个小区的销售情况
print(get_data('第五郡五号地'))
df_wu=df[df['title'].apply(lambda x: '第五郡' in x and '公寓' not in x)]
df_wu=df_wu.sort_values(by='dealDate')
df_wu=df_wu.reset_index(drop=True)
func=lambda x:str((x.year)*100+(x.month))[0:4]+'-'+str((x.year)*100+(x.month))[4:]
df_wu['month']=df_wu['dealDate'].map(func)
df_wu['title'].groupby(df_wu['month']).count()
# df[df['title']=='金海花园西园'][['title']].count()

第五郡五号地 甘井子 第五郡 2006


month
2012-08     1
2015-12     1
2016-03     1
2016-05     1
2016-07     2
2016-08     6
2016-09     2
2016-10     6
2016-11     4
2016-12     5
2017-01     5
2017-02     6
2017-03     9
2017-04    11
2017-05     7
2017-06     2
2017-07     3
2017-08     5
2017-09     1
2017-11     2
2017-12     3
2018-01     1
2018-02     2
2018-03     4
2018-04     6
2018-05     1
2018-06     2
2018-07     5
2018-08     1
2018-09     5
2018-10     3
2018-11     2
2018-12     4
2019-01     3
2019-02     2
2019-03     6
2019-04     6
2019-05    10
2019-06     5
2019-07     6
2019-09     7
2019-11     5
2019-12     4
2020-01     3
2020-04     3
2020-05     3
2020-06     3
2020-07     4
2020-08     2
2020-09     4
2020-10     1
2020-11     5
2020-12     4
2021-01     2
2021-02    10
2021-03    11
2021-04    13
2021-05     5
Name: title, dtype: int64

In [8]:
#找出小区每年均价，按月标本太少，不太准确
df_wu['unitPrice'].groupby(df_wu['dealDate'].map(lambda x:x.year)).describe()

,count,mean,std,min,25%,50%,75%,max
dealDate,,,,,,,,
2012,1.0,17582.000000,NaN,17582.0,17582.00,17582.0,17582.00,17582.0
2015,1.0,24522.000000,NaN,24522.0,24522.00,24522.0,24522.00,24522.0
2016,27.0,17582.703704,2763.473634,13386.0,16190.00,17500.0,18279.50,28283.0
2017,54.0,19862.870370,4539.810802,14289.0,16939.00,18647.5,20932.00,36302.0
2018,36.0,25159.305556,4473.042920,19848.0,22522.75,24292.0,25636.00,43112.0
2019,54.0,27629.703704,5508.084438,20407.0,24503.75,25420.5,29598.50,46702.0
2020,32.0,28920.562500,6289.397489,19075.0,25262.75,26917.0,30312.25,46436.0
2021,41.0,27453.560976,4921.531098,21143.0,25320.00,26800.0,28270.00,50839.0


In [9]:
#找出小区的成交周期
df_wu['dealCycle'].groupby(df_wu['dealDate'].apply(lambda x:x.year)).describe()

,count,mean,std,min,25%,50%,75%,max
dealDate,,,,,,,,
2012,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
2015,1.0,32.000000,NaN,32.0,32.00,32.0,32.00,32.0
2016,27.0,153.185185,103.315066,6.0,72.00,133.0,233.50,419.0
2017,54.0,277.500000,191.430144,1.0,102.50,270.5,435.50,617.0
2018,36.0,155.861111,148.745449,6.0,33.50,119.5,209.50,605.0
2019,54.0,222.555556,275.108669,2.0,50.00,132.0,268.25,1343.0
2020,32.0,282.656250,253.384523,3.0,112.25,174.0,415.25,886.0
2021,41.0,251.195122,297.797936,1.0,65.00,141.0,281.00,1481.0


In [10]:
#找出房子里只有一天成交周期的房子
df[(df['dealCycle'].map(lambda x:x==1)) & (df['dealDate'].apply(lambda x:x.year)==2021)].head()

,title,room,area,dealDate,totalPrice,unitPrice,dealCycle,guaPai,seller
76,中庚香海金鼎回迁楼,2室1厅,68.00,2021-04-28,36.0,5221.0,1,36.0,于丽华
77,中庚香海金鼎回迁楼,1室1厅,61.42,2021-03-04,21.0,3420.0,1,21.5,何新玉
961,高洁巷,2室1厅,51.10,2021-04-20,62.0,12134.0,1,62.5,刘彦秀
1566,MINI印象,1室0厅,50.80,2021-03-27,84.0,16536.0,1,86.0,周浩
2236,中海凤凰熙岸,2室2厅,73.80,2021-02-21,109.0,14770.0,1,110.0,付霞


In [11]:
#备用功能        
# list=[]
# for i in df['title'].drop_duplicates():
#     g=get_data(i)
#     if g is not None:
#         list.append(g)

# %run main.py
#df.to_json(path_or_buf='data/test.json',orient='index',force_ascii=False)
#检查area数据
# for i in range(0,len(df)):
#     result=df.iloc[i]
#     if re.search(r'\d+(\.\d+)?(平米)',result[2]) is None:
#         print(df.iloc[i])
#         df.drop([i])
# #检查挂牌数据
# for i in range(0,len(df)):
#     result=df.iloc[i]
#     if re.search('挂牌*',result[7]) is None:
#         print(str(i))
        # df.drop([i])

In [12]:
#根据已有数据算全部小区
dd=df['unitPrice'].groupby(df['dealDate'].apply(lambda x:x.year)).describe()
dd[dd.index>2015]

,count,mean,std,min,25%,50%,75%,max
dealDate,,,,,,,,
2016,8351.0,10152.072686,4199.410162,3050.0,7881.0,9289.0,11365.50,128409.0
2017,13557.0,10934.225566,4429.893607,1834.0,8187.0,9959.0,12419.00,105216.0
2018,13644.0,12118.601730,4939.022594,3487.0,8770.0,11038.5,14032.25,94424.0
2019,17124.0,12918.877891,5224.095472,2008.0,9225.0,12087.0,14884.00,82069.0
2020,19493.0,13730.577695,5996.160904,1705.0,9584.0,12888.0,15890.00,65225.0
2021,11516.0,14950.965353,6138.918913,2.0,10427.5,14112.0,17453.25,61788.0


In [13]:
df.head(1)

,title,room,area,dealDate,totalPrice,unitPrice,dealCycle,guaPai,seller
0,金园新村,2室1厅,90.84,2021-04-29,80.0,8807.0,259,82.0,王冰


In [19]:
test=set(df['title'])
type(test)
for i in test:
    print(i)

39中顺阳街
中盛秀山丽水
世纪英伦
谷歌里
通用阳光丽景
金石小区
黄河路 辽师商圈
桂林小区
雍景国际
小孤山中里
迎春街（旅顺口）
39中安达大厦公寓楼
远洋荣域B1区
中海紫御观邸四期
39中学士街
中盛家园
招商海德公园
桃园小区
星海街
亿达春田馥园
双兴北巷
新星绿城三期
水果新城四期
亿锋广场二期
小平岛E西组团
胜鼎景山
连桥巷
21中天兴花园一期
秀月小区
理工文萃轩
中航国际广场二期
亿达普罗旺斯二期
中冶蓝城
红星海世界观二期视界
盐岛湖
嘉安边防家园
39中开源巷
南沙
红星海世界观一期艺墅
39中民建街
双兴财富长街（大菜市商圈）
良运缤纷四季
安邦北湾
千山心城
龙畔金泉五期K2区
双全街
诚泰文教城
运达诺维溪谷四期
新辰里
靠山巷
新辉小区
大华爱琴湾
金盛家园
亿达国际新城
五二三厂
观山蝶恋花
金二小学南
枫合万嘉
民生松竹苑
小平岛亲海园
欣家园一期
红星海五期岚谷
瑞仕尚城二期
桃源小学
21中广电中心
东城天下
保利西山林语
和兴家园
职工街（三八广场）
商城花园
三鼎春天B区
前城国际二期
中航华府
金苑玉庭小区
澳东世纪
南石源居
兴达依山居
金州福佳新天地
东城国际
远洋荣域B2区
世茂寰海城
艾维尼小城
远洋荣域A2区
芝麻街
招商兰溪谷北区
台扩小区
恒大明秀庄园
北府花园
百年港湾
春柳中心
山河御景
海韵山庄
小平岛
红星海二期视界
哈佛映像
红星海郡墅二号
北京公园
金馨园一区
龙湖星海彼岸
因特摩尔
山东路
天誉湾
桃山小区
万达华府二期
鹿港小镇
阳光驿城
润德广场
国宝星月湾
39中安阳街
春江花园
华东路 金三角商圈
丽湾海景
海尚明珠
宏都筑景Ⅰ期
双兴北巷（大菜市商圈）
丰源小区
新青年金洋合院（金洋小区）
绿波花园
王家桥小区
锦泉南园
弘基二期
嘉和广场
聚美东湾
红星海世界观三期视界
宏孚街
观山满庭芳
开建阳光馨苑
金连领地
桃花源小区
裕和苑小区
悦丽海湾
39中南山路
春园里
乐活源
长春花园
锦泉源D1区
辰熙星海国际
格林翠庭
中铁诺德滨海花园
同心街（昆明街）
青松南里
万科莫亚小镇
保利天禧
城市印象
花溪镇
红星海三期境界
文荟小区
泡崖康顺园
华南沃尔玛六一路
古城乙区
北欧假日
融创海逸长洲一期
松峪里一期
新星绿城二期
鲁能泰山7号一期
万科城
泉水友好园
南山路小区

In [5]:
import json
from pandas import DataFrame
def get_xiaoqu(n1,n2):
    list = []
    for line in open('data/lianjia.json', 'r', encoding='utf-8'):
        list.append(json.loads(line))

    df = DataFrame(list)
    item = set(df['title'])
    print(item)
    # item=item[n1,n2]
    # return item

get_xiaoqu(0,1)

{'新青年金洋合院', '澳东世纪', '大华锦绣华城一期', '金海花园西园', '金苑玉庭小区', '汇利嘉园', '德泰时代广场', '山海一家三期', '亿诚御景湾三期', '春和小区', '黑石礁', '枫丹丽城', '黄河路', '金润小区B区', '高民里', '五二三厂', '金宸国际', '阳光月秀', '八十中周家街', '新华街', '亿达春田馥园', '泡崖三区', '三寰融创城北园', '叠翠山庄', '新月里', '长利巷', '万国宫馆三期', '新华小区', '运达诺维溪谷一期', '东城天下', '欣家园二期', '职工街 民主广场商圈', '红星海三期视界', '达赫山庄', '虹港路（机场商圈）', '红星海世界观珑筑', '解放路 青云街商圈', '观山蝶恋花', '一方天鹅湖', '书香府第一期', '高城花园', '金海国际花园', '红星海世界观一期艺墅', '和平小区', '金湾山城', '鹏辉新世纪三期', '莲花小区', '泡崖五区', '虹港新居', '保亿丽景山', '福泉路小区', '水映西山', '憬园河畔', '幸福e家三期', '久寿河农贸市场', '金洋小区', '众益阳光海岸', '加州洋房', '大连明珠', '古耕国际酒店公寓', '高云街', '生态城', '林茂小区', '锦虹北园', '金色阳光', '虹桥干休所', '金海蓝湾', '中航国际广场二期', '翡翠春城', '八十中东纬路', '悦丽海湾四期', '新希望家园', '尚都家园', '锦霞北园', '凌源街', '软景E居', '碧海园', '长江路 中山友好商圈', '逸林派', '岭秀滨城悦和西园', '金连佳地', '79中学区广电中心', '三鼎春天A区', '远洋时代城四期', '新华小区(西岗)', '绿波花园', '格林翠庭', '御府山海观', '谷歌里', '泡崖新六区', '中航华府', '星台花园一期', '万科金域华府', '中铁诺德滨海花园B区', '佳兆业中心', '亿锋广场三期', '万成御园', '中国铁建国滨苑', '花香维也纳', '居家西园小区', '连大文润金宸一二期', '壹品漫谷', '华东路 山东路商圈', '颐莲家园', '观山悦', '39中白兰街（南山商圈）', '顺达温泉花

In [9]:
list=[]
for line in open('data/lianjia.json', 'r', encoding='utf-8'):
    list.append(json.loads(line))

df = DataFrame(list)
item = set(df['title'])
item[0]

TypeError: 'set' object is not subscriptable

In [13]:
l=[x for x in item]



TypeError: list indices must be integers or slices, not tuple

In [16]:
l[0:2]

['新青年金洋合院', '澳东世纪']